In [2]:
review ={}
import json
with open('pos_review.json','r',encoding = 'utf-8') as fp:
    review.update(json.load(fp))

In [3]:
texts = []
for home_id,rev_lst in review.items():
    for rev in rev_lst:
        for sentence in rev[1] :
            texts.append(sentence)

In [5]:
import gensim
from gensim import corpora, models
from gensim.models import CoherenceModel

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [6]:
dic = corpora.Dictionary(texts)

In [10]:
dic.save('dic_review.dict')

In [11]:
test_dic = open('dic_review.dict','r')

In [12]:
corpus = [dic.doc2bow(text) for text in texts]

In [13]:
corpora.MmCorpus.serialize('corpus_review.mm', corpus)  # store to disk, for later use

In [14]:
print(corpus)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### gensim.models.ldamodel.LdaModel parameter

- corpus: the corpus used to train the topic model.

- id2word: a dictionary that maps word numerical identifiers to word strings (see e.g. this tutorial for an explanation). In our case, this is handled automatically by the corpus class.

- alpha: the topic distribution. If we set it to 'auto', then we tell the algorithm to determine the topic proportions automatically.

- num_topics: how many topics the algorithm should find.

- passes: how many times the algorithm should go through the corpus. If you are using a large corpus (e.g. Wikipedia), then this can be set to 1, but for a small corpus (e.g. the reviews) then it should be a larger value.

In [ ]:
%%time
ldamodel = gensim.models.ldamodel.LdaModel(
                        corpus, 
                        num_topics=10, 
                        id2word = dic, 
                        passes=1
                        ) #LDA model 생성

In [16]:
dic_review = {}
dic_review.update(dic)

In [18]:
with open('dic_review.json','w',encoding='utf-8') as fp:
    json.dump(dic_review,fp,ensure_ascii=False)

In [ ]:
ldamodel.show_topics(num_words=10) #토픽을 구성하는 주요단어들

In [ ]:
# Compute Perplexity 
print('\nPerplexity: ', ldamodel.log_perplexity(corpus_vector))  # a measure of how good the model is. lower the better.

In [ ]:
# Visualize the topics

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus_vector, dic)
vis